## Machine Learning


#### Μέλη Ομάδας:
- Ευάγγελος Λευτάκης : 3200093
- Ρέα Σκλήκα : 3210192 
- Σοφία Σωτηρίου : 3210181

### Dependencies

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn

### 1st Part

First We Prepare the train, dev and test data and create a binary representation of it.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

(temp_x_train_imdb, temp_y_train_imdb), (x_test_imdb, y_test_imdb) = tf.keras.datasets.imdb.load_data()

split_index = int(0.8 * len(temp_x_train_imdb))  # 80% for training, 20% for dev

x_train_imdb, y_train_imdb = temp_x_train_imdb[:split_index], temp_y_train_imdb[:split_index]
x_dev_imdb, y_dev_imdb = temp_x_train_imdb[split_index:], temp_y_train_imdb[split_index:]

word_index = tf.keras.datasets.imdb.get_word_index()
index2word = dict((i + 3, word) for (word, i) in word_index.items())
index2word[0] = '[pad]'
index2word[1] = '[bos]'
index2word[2] = '[oov]'
x_train_imdb = np.array([' '.join([index2word[idx] for idx in text]) for text in x_train_imdb])
x_dev_imdb = np.array([' '.join([index2word[idx] for idx in text]) for text in x_dev_imdb])
x_test_imdb = np.array([' '.join([index2word[idx] for idx in text]) for text in x_test_imdb])

binary_vectorizer = CountVectorizer(binary=True, min_df=100)
x_train_imdb_binary = binary_vectorizer.fit_transform(x_train_imdb)
x_dev_imdb_binary = binary_vectorizer.transform(x_dev_imdb)
x_test_imdb_binary = binary_vectorizer.transform(x_test_imdb)
print(
    'Vocabulary size:', len(binary_vectorizer.vocabulary_)
)

Vocabulary size: 3216
